In [1]:
from data_loader import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack


In [2]:
train = load_train_data('data/train.csv', is_df = False)
test = load_test_data('data/test.csv', 'data/test_labels.csv', is_df = False)

/Users/caojingchen/UCI/projects/data_mining/toxic/data_loader.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  return train_df['id'].tolist(), train_df['comment_text'].tolist(), train_df.iloc[:, 2:].as_matrix(),\
/Users/caojingchen/UCI/projects/data_mining/toxic/data_loader.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  ), valid_df.iloc[:, 2:].as_matrix()
/Users/caojingchen/UCI/projects/data_mining/toxic/data_loader.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  return data_frame['id'].tolist(), data_frame['comment_text'].tolist(), label_frame.iloc[:, 1:].as_matrix()


In [3]:
print(test[2][2])

[0 0 0 0 0 0]


In [4]:
train_text, train_class = train[1], train[2]
val_text, val_class = train[4], train[5]
test_text, test_class = test[1], test[2]

In [5]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(train_text + val_text + test_text)
train_word_features = word_vectorizer.transform(train_text)
val_word_features = word_vectorizer.transform(val_text)
test_word_features = word_vectorizer.transform(test_text)

In [ ]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(train_text + val_text + test_text)
train_char_features = char_vectorizer.transform(train_text)
val_char_features = char_vectorizer.transform(val_text)
test_char_features = char_vectorizer.transform(test_text)

In [6]:
# train_features = hstack([train_char_features, train_word_features])
# val_features = hstack([val_char_features, val_word_features])
# test_features = hstack([test_char_features, test_word_features])
train_features = hstack([train_word_features])
val_features = hstack([val_word_features])
test_features = hstack([test_word_features])

In [7]:
print(train_features.shape)
print(train_class.shape)
print(test_features.shape)
print(test_class.shape)

(143645, 10000)
(143645, 6)
(63978, 10000)
(63978, 6)


In [18]:
scores = []
for col in range(len(train_class[0])):
    train_target = train_class[:,col]
    val_target = val_class[:,col]
    test_target = test_class[:,col]
    
    lr = LogisticRegression(C=0.1, solver='sag')
    lr.fit(train_features, train_target)
    print(lr.score(test_features, test_target))
#     score.append(classifier.score(test_features, test_target))


0.9372127918972146
0.9934352433649067
0.9627059301634937
0.9967019913095126
0.9585326205883272
0.9889336959579855


In [ ]:
print(lr.score(test_features, test_class))
print(lr.score(val_features, val_class))

In [8]:
dt = DecisionTreeClassifier(max_depth=2000, min_samples_split=10, min_samples_leaf=10, max_leaf_nodes=160)
dt.fit(train_features, train_class)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2000,
            max_features=None, max_leaf_nodes=160,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=10,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [11]:
print(dt.score(test_features, test_class))
print(dt.score(val_features, val_class))

0.8755040795273376
0.9144166771317342


In [12]:
rf = RandomForestClassifier(n_estimators=10, max_depth=2000, min_samples_split=10, min_samples_leaf=10, max_leaf_nodes=150)
rf.fit(train_features, train_class)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2000, max_features='auto', max_leaf_nodes=150,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
print(rf.score(test_features, test_class))
print(rf.score(val_features, val_class))

0.8961986933008221
0.9127841265854577


In [ ]:
def getAnswer(model, s_input):
    ret = model.predict(hstack([char_vectorizer.transform([s_input]),word_vectorizer.transform([s_input])]))

In [ ]:
print(getAnswer(rf, "Fuck you bitch!!"))